In [4]:
using Pipe

parse_pic(str) = begin
    lines = split(str, "\n")
    m = match(r"^Tile (\d+):$", lines[1])
    num = parse(Int, m[1])
    arr = fill(' ', (length(lines[2]), length(lines[2])))
    
    for (i, row) in enumerate(collect.(lines[2:end]))
        for (j, c) in enumerate(row)
            arr[i,j] = c
        end
    end
    
    num => arr
end

get_input(file) = open(file) do f
    pics = split(read(f, String), "\n\n")
    Dict(parse_pic.(pics))
end

get_input (generic function with 1 method)

In [5]:
pics = get_input("20.dat");

In [6]:
pic_sides(p) = (p[1:10,1], p[1:10,10], p[1,1:10], p[10,1:10], reverse(p[1:10,1]), reverse(p[1:10,10]), reverse(p[1,1:10]), reverse(p[10,1:10]))

pic_sides (generic function with 1 method)

In [7]:
function all_sides(pics)
    sides_cache = Dict{Array{Char, 1}, Array{Int, 1}}()
    
    for (k, v) in pics
        ps = pic_sides(v)
        for s in ps
            if s in keys(sides_cache)
                push!(sides_cache[s], k)
            else
                sides_cache[s] = [k]
            end
        end
    end
    
    sides_cache
end

function find_corners(sides, pics)
    ok = []
    for (k, v) in pics
        s = pic_sides(v)
        num_neighbors = 0
        for side in s
            if length(sides[side]) > 1
                num_neighbors += 1
            end
        end
        if num_neighbors == 4 # 4 cus reverses count too
            push!(ok, k)
        end
    end
    ok
end

find_corners (generic function with 1 method)

In [8]:
find_corners(all_sides(pics), pics)

4-element Array{Any,1}:
 2287
 3433
 3461
 3083

In [9]:
function join_images(pics; instead=nothing)
    as = all_sides(pics)
    corners = find_corners(as, pics)
    tl = instead==nothing ? pics[corners[1]] : pics[instead] # top-left corner
    idx = instead == nothing ? corners[1] : instead
    
    toprow(p) = p[1,1:10]
    leftcol(p) = p[1:10,1]
    bottomrow(p) = p[10,1:10]
    rightcol(p) = p[1:10,10]
    
    sz::Int = sqrt(length(pics))
    
    @show sz
    
    for i in 0:3
        r = rotr90(tl, i)
        trl = length(as[toprow(r)])
        lcl = length(as[leftcol(r)])
        
        if trl == lcl == 1
            tl = r
        end
    end
    
    left = tl
    lidx = idx
    
    layout::Array{Int, 2} = zeros((sz, sz))
    layout[1,1] = idx
    
    dis_layout::Array{Array{Char, 2}, 2} = fill(zeros((10,10)), (sz, sz))
    dis_layout[1,1] = tl
    
    function findfits(pic, left, f)
        for i in 0:3
            r = rotr90(pic, i)
            if f(r) == left
                return copy(r)
            end
            if f(reverse(r;dims=1)) == left
                return copy(reverse(r;dims=1))
            end
        end
    end
    
    tlidx = lidx
    
    for j in 1:sz
        for i in 2:sz
            #display(left)
            nl = filter(x -> x != lidx, as[rightcol(left)])[1]
            layout[j,i] = nl
            left = findfits(pics[nl], rightcol(left), leftcol)
            dis_layout[j,i] = left
            lidx = nl
        end
        
        if j == sz
            break
        end

        new = @pipe as[bottomrow(tl)] |> filter(x -> x != tlidx, _) |> first
        @show new
        
        layout[j + 1,1] = new
        ntl = pics[new]
        left = findfits(ntl, bottomrow(tl), toprow)
        dis_layout[j + 1,1] = left
        lidx = new
        tlidx = new
        tl = left
    end
    layout, dis_layout
end

join_images (generic function with 1 method)

In [10]:
layout, dis = join_images(pics)

sz = 12
new = 1567
new = 2221
new = 3019
new = 1283
new = 1579
new = 3529
new = 1361
new = 2803
new = 1051
new = 2719
new = 3433


([2287 1093 … 1871 3083; 1567 3463 … 2791 3677; … ; 2719 3191 … 1217 2141; 3433 3221 … 3061 3461], [['.' '.' … '.' '#'; '.' '.' … '#' '#'; … ; '#' '.' … '.' '#'; '#' '.' … '.' '.'] ['#' '#' … '.' '.'; '#' '#' … '.' '#'; … ; '#' '.' … '.' '.'; '.' '.' … '#' '.'] … ['.' '#' … '#' '.'; '#' '.' … '.' '#'; … ; '#' '.' … '.' '.'; '.' '.' … '.' '#'] ['.' '.' … '#' '#'; '#' '.' … '#' '#'; … ; '.' '.' … '.' '#'; '#' '#' … '#' '#']; ['#' '.' … '.' '.'; '.' '.' … '#' '.'; … ; '.' '.' … '#' '#'; '.' '#' … '#' '#'] ['.' '.' … '#' '.'; '.' '.' … '.' '.'; … ; '#' '.' … '.' '#'; '#' '#' … '#' '#'] … ['.' '.' … '.' '#'; '.' '.' … '#' '#'; … ; '.' '.' … '.' '.'; '#' '.' … '.' '#'] ['#' '#' … '#' '#'; '#' '.' … '#' '.'; … ; '.' '#' … '.' '#'; '#' '.' … '#' '.']; … ; ['.' '#' … '.' '#'; '.' '.' … '.' '#'; … ; '.' '.' … '.' '#'; '#' '.' … '#' '#'] ['#' '#' … '#' '#'; '#' '.' … '.' '.'; … ; '#' '.' … '#' '.'; '#' '.' … '.' '#'] … ['.' '#' … '#' '.'; '.' '.' … '#' '.'; … ; '.' '.' … '.' '#'; '#' '#' … '.' '#

In [11]:
trim_border(arr) = arr[2:9,2:9]

trim_border (generic function with 1 method)

In [12]:
trimmed = trim_border.(dis)

12×12 Array{Array{Char,2},2}:
 ['.' '#' … '.' '#'; '.' '.' … '.' '.'; … ; '#' '.' … '.' '#'; '.' '#' … '#' '.']  …  ['.' '.' … '.' '#'; '.' '.' … '.' '.'; … ; '.' '.' … '.' '.'; '.' '.' … '.' '.']
 ['.' '.' … '.' '#'; '.' '.' … '.' '.'; … ; '.' '.' … '#' '#'; '.' '#' … '#' '#']     ['.' '.' … '.' '#'; '.' '#' … '.' '.'; … ; '.' '.' … '#' '.'; '#' '#' … '.' '.']
 ['.' '#' … '#' '.'; '.' '.' … '.' '#'; … ; '.' '.' … '.' '.'; '.' '.' … '.' '#']     ['.' '.' … '.' '.'; '#' '#' … '#' '.'; … ; '.' '.' … '.' '.'; '#' '#' … '#' '.']
 ['.' '.' … '.' '.'; '.' '.' … '#' '.'; … ; '.' '#' … '.' '.'; '.' '.' … '#' '#']     ['.' '.' … '.' '.'; '#' '#' … '#' '#'; … ; '.' '.' … '.' '.'; '.' '#' … '.' '.']
 ['.' '.' … '.' '#'; '#' '.' … '#' '.'; … ; '#' '#' … '#' '.'; '.' '.' … '.' '#']     ['.' '.' … '.' '#'; '#' '#' … '#' '.'; … ; '.' '.' … '.' '.'; '.' '.' … '.' '.']
 ['#' '.' … '#' '.'; '#' '.' … '.' '.'; … ; '.' '#' … '.' '.'; '.' '#' … '.' '.']  …  ['#' '#' … '.' '.'; '#' '#' … '#' '.'; … ; '.' '.

In [13]:
function join_all(pics)
    sx::Int = sqrt(length(pics))
    sp::Int = sqrt(length(pics[1,1]))
    sz::Int = sx * sp
    
    final::Array{Char, 2} = fill(' ', (sz, sz))
    
    for oj in 0:(sx - 1)
        for oi in 0:(sx - 1)
            for ij in 1:sp
                for ii in 1:sp
                    final[oj * sp + ij, oi * sp + ii] = pics[oj + 1,oi + 1][ij,ii]
                end
            end
        end
    end
    
    final
end

join_all (generic function with 1 method)

In [14]:
joined = join_all(trimmed)

96×96 Array{Char,2}:
 '.'  '#'  '.'  '.'  '.'  '#'  '.'  '#'  …  '.'  '.'  '.'  '.'  '#'  '.'  '#'
 '.'  '.'  '.'  '.'  '.'  '.'  '.'  '.'     '.'  '#'  '.'  '.'  '.'  '.'  '.'
 '.'  '.'  '#'  '.'  '.'  '#'  '#'  '#'     '#'  '.'  '.'  '.'  '.'  '.'  '.'
 '#'  '.'  '#'  '#'  '.'  '.'  '.'  '#'     '.'  '#'  '.'  '.'  '.'  '.'  '.'
 '.'  '.'  '.'  '#'  '.'  '#'  '.'  '.'     '#'  '.'  '.'  '.'  '#'  '.'  '.'
 '#'  '#'  '.'  '.'  '.'  '.'  '.'  '#'  …  '.'  '.'  '#'  '.'  '.'  '.'  '#'
 '#'  '.'  '#'  '.'  '#'  '.'  '.'  '#'     '.'  '#'  '.'  '#'  '#'  '.'  '.'
 '.'  '#'  '.'  '.'  '#'  '.'  '#'  '.'     '.'  '.'  '#'  '.'  '.'  '.'  '.'
 '.'  '.'  '.'  '.'  '.'  '.'  '.'  '#'     '.'  '#'  '.'  '#'  '.'  '.'  '#'
 '.'  '.'  '.'  '.'  '.'  '.'  '.'  '.'     '#'  '.'  '.'  '.'  '.'  '.'  '.'
 '.'  '.'  '#'  '.'  '.'  '.'  '.'  '.'  …  '.'  '.'  '.'  '#'  '.'  '.'  '#'
 '#'  '#'  '.'  '.'  '.'  '.'  '#'  '.'     '.'  '.'  '.'  '.'  '.'  '.'  '.'
 '.'  '.'  '#'  '#'  '.'  '.'  '.'  '.'    

In [15]:
tomat(a) = permutedims(reshape(hcat(a...), (length(a[1]), length(a))))

tomat (generic function with 1 method)

In [16]:
monster = ["                  # ",
           "#    ##    ##    ###",
           " #  #  #  #  #  #   "] |> x -> collect.(x)
monster_pounds = count(x -> x == '#', reduce(vcat, monster))

15

In [17]:
# 0-based indices (offset)
monster_outline = @pipe collect.(enumerate.(monster)) |> map.(x -> (x[1] - 1, x[2]), _) |> filter.(x -> x[2] == '#', _)

3-element Array{Array{Tuple{Int64,Char},1},1}:
 [(18, '#')]
 [(0, '#'), (5, '#'), (6, '#'), (11, '#'), (12, '#'), (17, '#'), (18, '#'), (19, '#')]
 [(1, '#'), (4, '#'), (7, '#'), (10, '#'), (13, '#'), (16, '#')]

In [23]:
function monster_at(pics, j, i)
    sj, si = size(pics)
    for (i_offset, row) in enumerate(monster_outline)
        for (j_offset, c) in row
            if i + i_offset - 1 > sj || j + j_offset > sj
                return false
            end
            if pics[j + j_offset, i + i_offset - 1] != c
                return false
            end
        end
    end
    return true
end

function find_monsters(joined)
    sz::Int = size(joined)[1]
    monsters = 0
    for j in 1:sz
        for i in 1:sz
            if monster_at(joined, j, i)
                monsters += 1
            end
        end
    end
    monsters
end

find_monsters (generic function with 1 method)

In [25]:
function find_all(joined)
    for i in 0:3
        r = rotr90(joined, i)
        found = find_monsters(r)
        if found > 0
            return found
        end

        r3 = reverse(r; dims=1)
        found = find_monsters(r3)
        if found > 0
            return found
        end
    end
end

find_all (generic function with 1 method)

In [29]:
count(x -> x == '#', joined) - find_all(joined) * 15

1993